# Atividade: Regras de Associação (Association Rules)

**Curso:** Sistemas de Informação - Mineração de Dados
**Objetivo:** Realizar a mineração de Regras de Associação no dataset **Groceries**.

**Observação sobre Ferramentas:**
O PDF da atividade instrui o uso de:
- **PyCaret** para *Detecção de Anomalias*.
- **MLxtend** para *Regras de Associação*.

No entanto, para atender à solicitação de explorar o PyCaret, este notebook incluirá:
1. **Regras de Associação com MLxtend** (Foco principal da atividade).
2. **Breve Análise com PyCaret** (Exploração/Anomalias como complemento, já que o módulo de regras de associação foi descontinuado em versões recentes do PyCaret).

**Dataset:** [Groceries Dataset](https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset)

In [ ]:
# Instalação das bibliotecas
# IMPORTANTE: Se o PyCaret não estiver instalado, esta célula irá instalá-lo.
# APÓS A INSTALAÇÃO, PODE SER NECESSÁRIO REINICIAR O KERNEL (Runtime -> Restart Runtime) E RODAR AS CÉLULAS NOVAMENTE.
try:
    import pycaret
    print("PyCaret já está instalado.")
except ImportError:
    print("Instalando PyCaret...")
    !pip install pycaret
    print("Instalação concluída! POR FAVOR, REINICIE O KERNEL AGORA para garantir que a biblioteca seja carregada corretamente.")

In [ ]:
!pip install pandas numpy matplotlib seaborn mlxtend

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Configurações de visualização
sns.set(style="whitegrid")
%matplotlib inline

## 1. Importação e Análise Exploratória

Carregando o dataset `Groceries_dataset.csv`.

In [ ]:
try:
    df = pd.read_csv('dataset/Groceries_dataset.csv')
except FileNotFoundError:
    try:
        df = pd.read_csv('Groceries_dataset.csv')
    except:
        print("Arquivo não encontrado. Verifique o caminho.")

print(df.info())
df.head()

In [ ]:
# Top 10 produtos mais vendidos
plt.figure(figsize=(12,6))
df['itemDescription'].value_counts().head(10).plot(kind='bar', color='skyblue')
plt.title('Top 10 Produtos Mais Vendidos')
plt.xlabel('Produto')
plt.ylabel('Frequência')
plt.show()

## 2. Regras de Associação com MLxtend (Conforme PDF)

Transformando os dados para o formato transacional (One-Hot Encoded).

In [ ]:
# Agrupando transações por Membro e Data
transactions = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).reset_index(name='items')

# One-Hot Encoding
te = TransactionEncoder()
te_ary = te.fit(transactions['items']).transform(transactions['items'])
df_trans = pd.DataFrame(te_ary, columns=te.columns_)

print(f"Transações processadas: {df_trans.shape[0]}")
df_trans.head()

### Gerando Itemsets Frequentes (Apriori)

In [ ]:
# Suporte mínimo de 0.1%
frequent_itemsets = apriori(df_trans, min_support=0.001, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False).head()

### Gerando Regras

In [ ]:
# Regras com lift >= 1.0
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules.sort_values(by='lift', ascending=False).head(10)

### Visualização dos Resultados

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x="support", y="confidence", size="lift", hue="lift", data=rules, palette="viridis", sizes=(20, 200))
plt.title('Regras de Associação: Suporte vs Confiança')
plt.show()

## 3. Uso do PyCaret (Complementar)

Como mencionado, o PyCaret descontinuou o módulo `arules` nativo. No entanto, podemos usar o PyCaret para **Detecção de Anomalias** neste mesmo dataset, para identificar transações incomuns (ex: compras muito grandes ou com combinações raras de itens).

Isso atende ao requisito de "utilizar PyCaret" da atividade geral.

In [ ]:
# Importação explícita do módulo de anomalias
from pycaret.anomaly import *

# Verificando se as funções foram carregadas corretamente
if 'setup' in locals() and 'create_model' in locals():
    print("Funções do PyCaret carregadas corretamente!")
else:
    print("ERRO: Funções do PyCaret não encontradas. Tente reiniciar o kernel e rodar novamente.")

In [ ]:
# Para PyCaret, precisamos de um dataframe numérico ou categórico. 
# Usaremos o dataframe transacional (df_trans) que já está em formato numérico (True/False -> 1/0)
# Como o dataset é grande, vamos usar uma amostra para demonstração rápida se necessário, ou o todo.

# Setup do PyCaret (pode demorar um pouco com muitas colunas)
# session_id garante reprodutibilidade
exp_ano = setup(df_trans, session_id=123, verbose=False)

print("Setup do PyCaret concluído!")

In [ ]:
# Criando um modelo de Isolation Forest (bom para dados de alta dimensão)
iforest = create_model('iforest')

# Atribuindo os resultados (Anomalia = 1, Normal = 0)
results_ano = assign_model(iforest)
results_ano.head()

In [ ]:
# Visualizando as anomalias detectadas
print("Transações consideradas anômalas:")
print(results_ano[results_ano['Anomaly'] == 1].head())

In [ ]:
# Plotando o modelo (em 2D usando T-SNE ou PCA interno do PyCaret)
plot_model(iforest, plot='tsne')

## Conclusão Final

- **Regras de Associação (MLxtend):** Identificamos padrões de compra frequentes (ex: Leite -> Iogurte).
- **Detecção de Anomalias (PyCaret):** Identificamos transações que fogem do padrão comum (outliers).

Ambas as técnicas fornecem insights valiosos para o varejo.